In [ ]:

import cv2
import numpy as np
import glob
# Chessboard settings (adjust based on your board)
chessboard_size = (10, 7)  # 9x6 grid intersections
square_size = 25  # in mm (for real-world scale, optional)

# Prepare object points (3D)
objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

objpoints = []  # 3D world points
imgpoints = []  # 2D image points

# Load calibration images
images = glob.glob("Calibration_Images/*.jpg")  # Only JPG  # Match all files # Path to saved images

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and show corners
        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('Corners', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Camera Calibration
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Save calibration results
np.savez("camera_calibration.npz", camera_matrix=camera_matrix, dist_coeffs=dist_coeffs)

print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs)

In [ ]:
total_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], camera_matrix, dist_coeffs)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    total_error += error
print("Mean Reprojection Error:", total_error / len(objpoints))

In [ ]:
# Load camera calibration parameters
calibration_data = np.load("camera_calibration.npz")
camera_matrix = calibration_data["camera_matrix"]
dist_coeffs = calibration_data["dist_coeffs"]

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import scipy.fftpack
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import os
import shutil

# Define folder name
frame_save_path = "captured_frames"

# Delete the folder if it exists and create a new one
if os.path.exists(frame_save_path):
    shutil.rmtree(frame_save_path)

os.makedirs(frame_save_path)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Initialize MediaPipe Pose for forearm reference
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Open full-screen webcam window
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

# Store angles over frames
finger_angles_over_time = {"calibrated": {"left": [], "right": []}, "uncalibrated": {"left": [], "right": []}}
finger_impact = {
    "calibrated": {"left": {"index_finger": [], "middle_finger": [], "ring_finger": [], "pinky_finger": [], "thumb": [], "wrist": []},
                   "right": {"index_finger": [], "middle_finger": [], "ring_finger": [], "pinky_finger": [], "thumb": [], "wrist": []}},
    "uncalibrated": {"left": {"index_finger": [], "middle_finger": [], "ring_finger": [], "pinky_finger": [], "thumb": [], "wrist": []},
                     "right": {"index_finger": [], "middle_finger": [], "ring_finger": [], "pinky_finger": [], "thumb": [], "wrist": []}}
}

# Define a function to calculate angles between three points in 3D
def calculate_angle(a, b, c):
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    cos_theta = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip to avoid errors
    return np.degrees(angle)

# Process video
frame_count = 0
start_time = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip and convert to RGB
    frame = cv2.flip(frame, 1)  # Correct left-right mirroring
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process uncalibrated frame
    results = hands.process(image_rgb)
    pose_results = pose.process(image_rgb)

    # Process calibrated frame
    calibrated_frame = cv2.undistort(frame, camera_matrix, dist_coeffs)
    calibrated_image_rgb = cv2.cvtColor(calibrated_frame, cv2.COLOR_BGR2RGB)
    calibrated_results = hands.process(calibrated_image_rgb)
    calibrated_pose_results = pose.process(calibrated_image_rgb)

    # Store data for both pipelines
    if results.multi_hand_landmarks and results.multi_handedness and pose_results.pose_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_label = "left" if handedness.classification[0].label.lower() == "left" else "right"
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            
            # Correct elbow landmark reference
            elbow_index = 14 if hand_label == "left" else 13
            elbow_landmark = pose_results.pose_landmarks.landmark[elbow_index]
            forearm_point = (elbow_landmark.x, elbow_landmark.y, elbow_landmark.z)
            
            angles = {
                'index_finger': calculate_angle(landmarks[5], landmarks[6], landmarks[7]),
                'middle_finger': calculate_angle(landmarks[9], landmarks[10], landmarks[11]),
                'ring_finger': calculate_angle(landmarks[13], landmarks[14], landmarks[15]),
                'pinky_finger': calculate_angle(landmarks[17], landmarks[18], landmarks[19]),
                'thumb': calculate_angle(landmarks[2], landmarks[3], landmarks[4]),
                'wrist': calculate_angle(forearm_point, landmarks[0], landmarks[12]) if forearm_point else None
            }
            for finger, angle in angles.items():
                if angle is not None:
                    finger_impact["uncalibrated"][hand_label][finger].append(angle)
            finger_angles_over_time["uncalibrated"][hand_label].append(angles)
            
            # Draw hand landmarks with coordinates
            for idx, lm in enumerate(hand_landmarks.landmark):
                cx, cy = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                cv2.putText(frame, f'{idx}', (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1, cv2.LINE_AA)
                cv2.circle(frame, (cx, cy), 6, (255, 0, 0), -1)
            
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Draw pose landmarks with the same style as hand landmarks
            for idx, lm in enumerate(pose_results.pose_landmarks.landmark):
                cx, cy = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                cv2.putText(frame, f'{idx}', (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1, cv2.LINE_AA)
                cv2.circle(frame, (cx, cy), 6, (255, 0, 0), -1)
            mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    if calibrated_results.multi_hand_landmarks and calibrated_results.multi_handedness and calibrated_pose_results.pose_landmarks:
        for calibrated_hand_landmarks, calibrated_handedness in zip(calibrated_results.multi_hand_landmarks, calibrated_results.multi_handedness):
            calibrated_hand_label = "left" if calibrated_handedness.classification[0].label.lower() == "left" else "right"
            calibrated_landmarks = [(lm.x, lm.y, lm.z) for lm in calibrated_hand_landmarks.landmark]
            
            # Correct elbow landmark reference
            calibrated_elbow_index = 14 if calibrated_hand_label == "left" else 13
            calibrated_elbow_landmark = calibrated_pose_results.pose_landmarks.landmark[calibrated_elbow_index]
            calibrated_forearm_point = (calibrated_elbow_landmark.x, calibrated_elbow_landmark.y, calibrated_elbow_landmark.z)
            
            calibrated_angles = {
                'index_finger': calculate_angle(calibrated_landmarks[5], calibrated_landmarks[6], calibrated_landmarks[7]),
                'middle_finger': calculate_angle(calibrated_landmarks[9], calibrated_landmarks[10], calibrated_landmarks[11]),
                'ring_finger': calculate_angle(calibrated_landmarks[13], calibrated_landmarks[14], calibrated_landmarks[15]),
                'pinky_finger': calculate_angle(calibrated_landmarks[17], calibrated_landmarks[18], calibrated_landmarks[19]),
                'thumb': calculate_angle(calibrated_landmarks[2], calibrated_landmarks[3], calibrated_landmarks[4]),
                'wrist': calculate_angle(calibrated_forearm_point, calibrated_landmarks[0], calibrated_landmarks[12]) if calibrated_forearm_point else None
            }
            for finger, angle in calibrated_angles.items():
                if angle is not None:
                    finger_impact["calibrated"][calibrated_hand_label][finger].append(angle)
            finger_angles_over_time["calibrated"][calibrated_hand_label].append(calibrated_angles)

    # Save the current frame
    cv2.imwrite(os.path.join(frame_save_path, f'frame_{frame_count}.jpg'), frame)
    
    # FPS Calculation
    frame_count += 1
    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time
    cv2.putText(frame, f'FPS: {fps:.2f}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('Hand Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Convert collected angles to DataFrames for frequency analysis
angles_df = {
    "calibrated": {hand: pd.DataFrame(finger_angles_over_time["calibrated"][hand]) for hand in ["left", "right"]},
    "uncalibrated": {hand: pd.DataFrame(finger_angles_over_time["uncalibrated"][hand]) for hand in ["left", "right"]}
}

# Finger impact analysis (range of motion)
impact_analysis = {
    "calibrated": {
        hand: {finger: max(values) - min(values) for finger, values in finger_impact["calibrated"][hand].items()}
        for hand in ["left", "right"]
    },
    "uncalibrated": {
        hand: {finger: max(values) - min(values) for finger, values in finger_impact["uncalibrated"][hand].items()}
        for hand in ["left", "right"]
    }
}
print("Finger Impact Analysis (Range of Motion in Degrees):", impact_analysis)

# Frequency analysis using Plotly subplots
colors = ['blue', 'green', 'red', 'purple', 'orange', 'brown']
for hand in ["left", "right"]:
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Calibrated", "Uncalibrated"))
    
    # Calibrated data
    df_calibrated = angles_df["calibrated"][hand]
    for idx, finger in enumerate(df_calibrated.columns):
        signal = df_calibrated[finger].values
        fft_values = np.abs(scipy.fftpack.fft(signal))[:len(signal)//2]
        freqs = np.fft.fftfreq(len(signal))[:len(signal)//2]
        fig.add_trace(go.Scatter(x=freqs, y=fft_values, mode='lines', name=f"Calibrated {finger}", line=dict(color=colors[idx % len(colors)])), row=1, col=1)
    
    # Uncalibrated data
    df_uncalibrated = angles_df["uncalibrated"][hand]
    for idx, finger in enumerate(df_uncalibrated.columns):
        signal = df_uncalibrated[finger].values
        fft_values = np.abs(scipy.fftpack.fft(signal))[:len(signal)//2]
        freqs = np.fft.fftfreq(len(signal))[:len(signal)//2]
        fig.add_trace(go.Scatter(x=freqs, y=fft_values, mode='lines', name=f"Uncalibrated {finger}", line=dict(color=colors[idx % len(colors)])), row=1, col=2)
    
    fig.update_layout(title=f"{hand.capitalize()} Hand Finger Movement Frequency Analysis", xaxis_title="Frequency (Hz)", yaxis_title="Amplitude")
    fig.show()

# Joint angle visualization over time using Plotly subplots
for hand in ["left", "right"]:
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Calibrated", "Uncalibrated"))
    
    # Calibrated data
    df_calibrated = angles_df["calibrated"][hand]
    for idx, finger in enumerate(df_calibrated.columns):
        fig.add_trace(go.Scatter(x=df_calibrated.index, y=df_calibrated[finger], mode='lines+markers', name=f"Calibrated {finger}", line=dict(color=colors[idx % len(colors)])), row=1, col=1)
    
    # Uncalibrated data
    df_uncalibrated = angles_df["uncalibrated"][hand]
    for idx, finger in enumerate(df_uncalibrated.columns):
        fig.add_trace(go.Scatter(x=df_uncalibrated.index, y=df_uncalibrated[finger], mode='lines+markers', name=f"Uncalibrated {finger}", line=dict(color=colors[idx % len(colors)])), row=1, col=2)
    
    fig.update_layout(title=f"{hand.capitalize()} Hand Joint Angles Over Time", xaxis_title="Frame Number", yaxis_title="Angle (Degrees)")
    fig.show()

# Finger impact visualization using Plotly subplots
for hand in ["left", "right"]:
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Calibrated", "Uncalibrated"))
    
    # Calibrated data
    impact_calibrated = impact_analysis["calibrated"][hand]
    fig.add_trace(go.Bar(x=list(impact_calibrated.keys()), y=list(impact_calibrated.values()), name="Calibrated", marker=dict(color=colors[:len(impact_calibrated)])), row=1, col=1)
    
    # Uncalibrated data
    impact_uncalibrated = impact_analysis["uncalibrated"][hand]
    fig.add_trace(go.Bar(x=list(impact_uncalibrated.keys()), y=list(impact_uncalibrated.values()), name="Uncalibrated", marker=dict(color=colors[:len(impact_uncalibrated)])), row=1, col=2)
    
    fig.update_layout(title=f"{hand.capitalize()} Hand Finger and Wrist Impact Analysis", xaxis_title="Finger/Wrist", yaxis_title="Range of Motion (Degrees)")
    fig.show()



# Generate individual angle graphs with impact detection and analysis (side by side)
for hand in ["left", "right"]:
    for finger in angles_df["calibrated"][hand].columns:
        fig = make_subplots(rows=1, cols=2, subplot_titles=("Calibrated", "Uncalibrated"))
        
        # Calibrated data
        df_calibrated = angles_df["calibrated"][hand]
        fig.add_trace(go.Scatter(x=df_calibrated.index, y=df_calibrated[finger], mode='lines', name=f"Calibrated {finger}", line=dict(color='blue')), row=1, col=1)
        
        # Impact detection for calibrated data
        velocity_calibrated = np.diff(df_calibrated[finger])
        threshold =  15 # Adjust this threshold based on your data
        impact_frames_calibrated = np.where(np.abs(velocity_calibrated) > threshold)[0] + 1
        for impact in impact_frames_calibrated:
            fig.add_vline(x=impact, line=dict(color='red', width=2), row=1, col=1)
        
        # Uncalibrated data
        df_uncalibrated = angles_df["uncalibrated"][hand]
        fig.add_trace(go.Scatter(x=df_uncalibrated.index, y=df_uncalibrated[finger], mode='lines', name=f"Uncalibrated {finger}", line=dict(color='green')), row=1, col=2)
        
        
        
        